In [10]:
from utils.consts import *

from utils.utilities import (
    test, show_dep, get_data, get_nominal_subjects, get_prep, get_objects, get_root, _is_verb, get_aux,
    check_if_wh, get_csubj, get_amod, get_actual_subtree_for_finding_agg, get_subtree_list)

import pickle
import json
import random
import json
from IPython.display import Markdown, display
from pprint import pprint
import numpy as np

import spacy
PIPE = spacy.load('en_coref_lg')

In [2]:
data_file = "./data/dev_dataset.json"
data = get_data(data_file)

In [11]:
def get_list_of_que_with_num(dt):
    df = "./data/%s_dataset.json"%dt
    ldata = get_data(df)
    que_list = list()
    for each in ldata:
        doc = PIPE(each['question'])
        for tok in doc:
            if tok.pos_ in ["NUM"]:
                que_list.append(each)
                break
    return que_list

In [16]:
def dump_dataset(dt):
    df = "./data_with_num/%s_dataset.json"%dt
    ldata = get_list_of_que_with_num(dt)
    with open(df, 'w') as fp:
        json.dump(ldata, fp)

In [ ]:
dump_dataset('train')

In [18]:
len(ques)

5187

In [36]:
data[0]

{'phase': 1,
 'table_id': '1-10015132-11',
 'question': 'What position does the player who played for butler cc (ks) play?',
 'sql': {'sel': 'Position',
  'conds': [['School/Club Team', '=', 'Butler CC (KS)']],
  'agg': ''},
 'header': ['Player',
  'No.',
  'Nationality',
  'Position',
  'Years in Toronto',
  'School/Club Team']}

In [50]:
def get_info():
    for each_d in data[:10]:
        doc = PIPE(each_d['question'])
        first_root = get_root(doc)
        subjs = get_nominal_subjects(first_root)
    #     return subjs
        print("QUESTION : ", each_d['question'])
        print("ROOT : ", first_root)
        print("SELECT : ", each_d['sql']['sel'])
        print("SUBJECTS : ")
        pprint(subjs)
        show_dep(doc)

In [68]:
def get_obj_if_prep(nominal):
    # With this we are checking if there is following links
    # `prep` ----> `pobj`
    # this will only hold if the current nominal has amod attached to it.
    # TODO: NOT WORKING TAKE A LOOK AT THIS.
    has_amod = False
    for child in nominal.children:
        if child.dep_ in ["amod"]:
            has_amod = True
            break
    if has_amod:
        nprep = get_prep(nominal)
        if nprep is not None:
            npobj = get_objects(nprep)
            return npobj
    return None  
    
    
def get_verb_and_subj_pair(data_sample):
#     print("\n========================================================\n")
#     pprint(data_sample)
    # get the doc representation of the string
    doc = PIPE(data_sample['question'])
    # Find out the root of the given statement
    root = get_root(doc)
#     import pdb; pdb.set_trace()
    is_verb = _is_verb(root)
    aux = None
#     if not is_verb:
        # Some times the root can be a noun, in that case it'll always be associated to
        # an aux verb, and we gotta find it
    aux = get_aux(root)
    # If there is any aux found, we use it as our verb root
    if aux is not None and len(aux) > 0:
        root_verb = aux[0]
    else:
        # Else, we use our root as it is
        root_verb = root
    # Check if there is any csubj attached this means we have two wh nouns attached to
    # two different verbs one is asking and other qualifying.
    csubj_verb = get_csubj(root_verb)
    if csubj_verb is not None:
        root_verb = csubj_verb
#     print("ROOT : {}".format(root.text))
#     print("AUX : ")
#     pprint(aux)
    # nominals in this case is going to be a list of all the cols
    # in SELECT clause. If we find any nsubj attached to the root_verb we catch it here.
    nominals = get_nominal_subjects(root_verb)
    is_wh = False
    # If what we have found is not actually a noun but a question noun (wh-word)
    # TODO: In this case we may want to go on hunt for the actual nominal col.
    if len(nominals) != 0:
        is_wh = check_if_wh(nominals)
    if is_wh:
        # for now
        pass
#     print("SUBJ : ")
#     pprint(nominals)
    # Even after all this if there is no way the prog have found a nominal
    # find out if there is any prop attached to the verb.
    # If there is then we may look for prep_obj attached to it for the nominal.
    prep = None
    if len(nominals) == 0:
        prep = get_prep(root_verb)
#     print("PREP : ")
#     pprint(prep)
    pobj = None
    if prep is not None:
        pobj = get_objects(prep)
        nominals.append(pobj)
#     print("POBJ : ")
#     pprint(pobj)
#     show_dep(doc)
    return root_verb, nominals

In [69]:
count = 0
for each_d in data[:10]:
    try:
        print("\n============================================\n")
        r_v, sub_nom = get_verb_and_subj_pair(each_d)
        print("QUESTION : ", each_d['question'])
        print("ROOT : ", r_v)
        print("SELECT : ", each_d['sql']['sel'])
        print("SUBJECTS : ")
        pprint(sub_nom)
    except Exception:
        pass



QUESTION :  What position does the player who played for butler cc (ks) play?
ROOT :  does
SELECT :  Position
SUBJECTS : 
[position]


QUESTION :  How many schools did player number 3 play at?
ROOT :  did
SELECT :  School/Club Team
SUBJECTS : 
[schools]




QUESTION :  Who is the player that wears number 42?
ROOT :  is
SELECT :  Player
SUBJECTS : 
[player]


QUESTION :  What player played guard for toronto in 1996-97?
ROOT :  played
SELECT :  Player
SUBJECTS : 
[player]


QUESTION :  Who are all of the players on the Westchester High School club team?
ROOT :  are
SELECT :  Player
SUBJECTS : 
[Who]


QUESTION :  What school/club team is Amir Johnson on?
ROOT :  is
SELECT :  School/Club Team
SUBJECTS : 
[team]


QUESTION :  What are the total amount of numbers on the Toronto team in 2005-06?
ROOT :  are
SELECT :  No.
SUBJECTS : 
[amount]


QUESTION :  What are the total number of positions on the Toronto team in 2006-07?
ROOT :  are
SELECT :  Position
SUBJECTS : 
[number]


QUESTION : 

In [73]:
#Get sel col directly from nominal subjs
count = 0
for each_d in data:
    try:
        v, subs = get_verb_and_subj_pair(each_d)
        if len(subs) > 0 and str.lower(subs[0].text) in str.lower(each_d['sql']['sel']):
            count += 1
    except Exception:
        pass

print(count)

4047


In [46]:
# Get sel col directly from nominal subjs
# count = 0
# for each_d in data:
#     try:
#         subs = get_info(each_d['question'])
#         if len(subs) > 0 and str.lower(subs[0].text) == str.lower(each_d['sql']['sel']):
#             count += 1
#     except Exception:
#         pass

# print(count)
# 2733

In [52]:
get_info()

QUESTION :  What position does the player who played for butler cc (ks) play?
ROOT :  does
SELECT :  Position
SUBJECTS : 
[position]


QUESTION :  How many schools did player number 3 play at?
ROOT :  play
SELECT :  School/Club Team
SUBJECTS : 
[]


QUESTION :  What school did player number 21 play for?
ROOT :  play
SELECT :  School/Club Team
SUBJECTS : 
[number]


QUESTION :  Who is the player that wears number 42?
ROOT :  is
SELECT :  Player
SUBJECTS : 
[player]


QUESTION :  What player played guard for toronto in 1996-97?
ROOT :  played
SELECT :  Player
SUBJECTS : 
[player]


QUESTION :  Who are all of the players on the Westchester High School club team?
ROOT :  are
SELECT :  Player
SUBJECTS : 
[Who]


QUESTION :  What school/club team is Amir Johnson on?
ROOT :  is
SELECT :  School/Club Team
SUBJECTS : 
[team]


QUESTION :  What are the total amount of numbers on the Toronto team in 2005-06?
ROOT :  are
SELECT :  No.
SUBJECTS : 
[amount]


QUESTION :  What are the total number of positions on the Toronto team in 2006-07?
ROOT :  are
SELECT :  Position
SUBJECTS : 
[number]


QUESTION :  What are the nationality of the players on the Fresno State school/club team?
ROOT :  are
SELECT :  Nationality
SUBJECTS : 
[nationality]


In [3]:
# This will work correctly on a doc object of a sentence
def get_root(sent_doc):
    # There is always a single root for a sentence.
    # This is just a safety guard
    roots = list()
    for word in sent_doc:
        if word.head == word:
            roots.append(word)
    if len(roots) != 1:
        return None
    return roots[0]

def get_compound(noun):
    compound_candidates = list()
    compound_candidates.append(noun)
    for child in noun.children:
        if (child.pos_ in NOUNS_POS or child.pos_ in DET_POS) and child.dep_ in COMPOUND_DEP:
            compound_candidates.extend(get_compound(child))
    return compound_candidates

def get_nominal_subjects(root):
    subject_list = list()
    for child in root.children:
        if child.pos_ in NOUNS_POS and child.dep_ in SUBJECT_DEP:
            subject_list.append(child)
    return subject_list

def get_appos(noun):
    # Assume that there will be only one appos
    for child in noun.children:
        if child.pos_ in NOUNS_POS and child.dep_ in APPOS:
            return child
    return None

def get_nsubjs(noun):
    conjs = list()
    conjs.append(noun)
    for child in noun.children:
        if child.pos_ in NOUNS_POS and child.dep_ in AND:
            conjs.extend(get_nsubjs(child))
    return conjs

def get_prep(noun):
    # assume there will be only one of these attached to the main noun
    prep = []
    for child in noun.children:
        if child.pos_ in PREP_POS and child.dep_ in PREPOSITIONAL_MOD:
            prep.append(child)
    return prep[-1]

def get_attributes(nominal_subj):
    appos_noun = get_appos(nominal_subj)
    if appos_noun is None:
        return get_nsubjs(nominal_subj)
    else:
        return get_nsubjs(appos_noun)
    
def get_objects(prep):
    for child in prep.children:
        if child.pos_ in NOUNS_POS and child.dep_ in OBJ_PREPOSITION:
            return child
    return None

def get_table_attr(main_obj):
    # Check if there, relative clause modifier
    relative_verb = None
    # check if 'relcl'
    for child in main_obj.children:
        if child.pos_ in VERB and child.dep_ in RELATIVE_CLAUSE_MOD:
            relative_verb = child
            break
    intermediate_prep = None
    # If there isn't then, check the for prep.
    if relative_verb is None:
        intermediate_prep = get_prep(main_obj)
    else:
        intermediate_prep = get_prep(relative_verb)
    objs = get_objects(intermediate_prep)
    return [main_obj, objs]

def get_attrs_n_objects(text):
    doc = PIPE(text)
    text_root = get_root(doc)
    attributes = get_attributes(get_nominal_subjects(text_root)[0])
    main_prep = get_prep(attributes[0])
    main_object = get_objects(main_prep)
    all_objects = get_table_attr(main_object)
    attributes = [get_compound(attr) for attr in attributes]
    all_objects = [get_compound(obj) for obj in all_objects]
    return attributes, all_objects

def join(list_attrs):
    return [" ".join([a.text for a in attrs]) for attrs in list_attrs]

def test(text):
    doc = PIPE(text)
    text_root = get_root(doc)
    print("root : {}".format(text_root))
    # I'll just be testing the first subjs
    nominal_subjs = get_nominal_subjects(text_root)
    print("nsubj : {}".format(nominal_subjs[0]))
    if len(nominal_subjs) > 1:
        print("There are more subjs are attached to ROOT")
    attrs = get_attributes(nominal_subjs[0])
    print("Attributes :")
    pprint([get_compound(attr) for attr in attrs])
    main_prep = get_prep(attrs[0])
    print("Prep : {}".format(main_prep))
    main_object = get_objects(main_prep)
    print("Objects : {}".format(main_object))
    all_objs = get_table_attr(main_object)
    print("All objs : ")
    pprint([get_compound(obj) for obj in all_objs])
    show_dep(doc)